# NLOS

## HHL Implementation

In [1]:
import math
import warnings
import numpy as np
import qiskit
from scipy.linalg import expm
from qiskit.circuit.library import phase_estimation, StatePreparation, ExactReciprocalGate
from qiskit.quantum_info import Statevector
from qiskit_aer import AerSimulator
from qiskit.compiler import transpile
from qiskit.result import Result
from qiskit.circuit import IfElseOp, QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.circuit.library import Initialize , UnitaryGate , QFTGate, ExactReciprocalGate

In [5]:
# todo: if A is not hermitian, do [[0, A], [A, 0]] to make it hermitian

def progress_callback(**kwargs):
    pass_name = kwargs['pass_'].name()
    count = kwargs['count']
    # The DAG circuit is an internal representation that can be inspected
    dag = kwargs['dag'] 
    # print(f"Pass {count}: {pass_name}, Operations: {dag.count_ops()}")

class HHL:
    """class that implements HHL algorithm"""

    def __init__(self, A: np.typing.NDArray, b: np.typing.NDArray, state_dim: int = -1, tol=1e-05):
        """A: operator matrix, b: matrix, n: number of qubits"""
        assert (len(A.shape) == 2 and len(b.shape) == 1), "Inconsistent dimension of A or b"
        assert (A.shape[0] == b.shape[0]), "shape of A does not match b"
        
        self.A = A.copy()
        self.b = b.copy()

        if (self.A.shape[0] != self.A.shape[1] or not np.allclose(self.A, self.A.conj().T, atol=tol)):
            if (self.A.shape[0] != self.A.shape[1]):
                print("A is not a square matrix")
            else:
                print("A is not Hermitian")
            self.A = np.block([[np.zeros((self.A.shape[0], self.A.shape[0])), self.A],
                               [self.A.conj().T, np.zeros((self.A.shape[1], self.A.shape[1]))]])
            
            print("A is evaluated as [[0, A], [AT, 0]]")
            print("-----------------------------------")
            self.b = np.block([b, np.zeros(A.shape[1])])
        
        # check if A is in shape 2^n * 2^n
        self.n_b = 0
        while (self.A.shape[0] > (1 << self.n_b)):
            self.n_b += 1

        
        
        # if A is not in shape 2^n, 2^n, pad A and b to the size
        # A padding: [[1]] -> [[1, 0], [0, a]] , b: [1] -> [1, 0]
        if (self.A.shape[0] != (1 << self.n_b)):
            print("A is not power of 2, pad to make A {0} * {0}".format((1 << self.n_b)))
            print("-----------------------------------")
            pad = (1 << self.n_b) - self.A.shape[0]
            before_pad = A.shape[0]

            # todo: avoid computing directly
            eigvals = np.linalg.eigvalsh(self.A)
            # set alpha to avoid eigvals to be 0
            # if eigval is zero, HHL blows up
            alpha = np.sqrt(np.abs(eigvals).min() * np.abs(eigvals).max())

            self.A = np.pad(self.A, ((0, pad), (0, pad)), "constant", constant_values=0)
            for i in range(before_pad, (1 << self.n_b)):
                self.A[i, i] = alpha
            self.b = np.pad(self.b, (0, (1 << self.n_b) - self.b.shape[0]), "constant", constant_values=0)

        self.tol = tol
        self.dim = (1 << self.n_b)

        if (not math.isclose(np.linalg.norm(self.b), 1, abs_tol=tol)):
            self.b = self.b.copy() / np.linalg.norm(self.b)
            print(f"b is not normalized, normalized to b = {self.b} during init")
            print("-----------------------------------")
        
        # C for making C/λ < 1
        eigvals = np.linalg.eigvalsh(self.A)
        self.C = np.abs(eigvals).min() * 0.9
        # todo: bound eigvals and avoid direct computation

        self.kappa = np.abs(eigvals).max() / np.abs(eigvals).min() # characteristic
        self.t = np.pi / np.abs(eigvals).max() # calculate t

        if (state_dim > 0):
            self.state_dim = state_dim
        else:
            self.state_dim = int(np.log2(self.kappa)) + 10

        self.U = UnitaryGate(expm(1j * self.t * self.A)) # U = e^(iAt), then convert to unitarygate


    
    def _create_register(self):
        """call this to initially create registers, only once"""
        b_reg = QuantumRegister(self.n_b, 'sys')
        c_reg = QuantumRegister(self.state_dim, 'phase')
        aux_reg = QuantumRegister(1, 'aux')
        classic_aux_reg = ClassicalRegister(1, 'classic_aux')
        result_reg = ClassicalRegister(self.n_b, 'classic_sys')
        return b_reg, c_reg, aux_reg, classic_aux_reg, result_reg


    def _prepare_state(self, circ: QuantumCircuit, b_reg: QuantumRegister):
        """add gate to mount b on registers"""
        # assert math.isclose(np.linalg.norm(self.b), 1, abs_tol=self.tol) # check if b is normalized
        init_gate = StatePreparation(Statevector(self.b))
        circ.append(init_gate, b_reg) # add gate for initalizing b
        circ.barrier() # separation from rest of the process


    def _controlled_rotation(self, circ, c_reg, aux_reg):
        """add controlled rotation block to circuit"""
        rot_gate = ExactReciprocalGate(self.state_dim, self.C * self.t / (2 * np.pi), neg_vals=True)
        circ.append(rot_gate, [*reversed(c_reg), *aux_reg])
    

    def _qpe(self, circ, b_reg, c_reg, inverse=False):
        """add QPE/IQPE gate to the circuit"""
        qpe_circ = phase_estimation(self.state_dim, self.U)
        if (inverse):
            qpe_circ = qpe_circ.inverse()
        circ.append(qpe_circ.to_gate(), [*c_reg, *b_reg]) # order may change


    def _measure(self, circ: QuantumCircuit, aux_reg: QuantumRegister, b_reg: QuantumRegister, classic_aux_reg: ClassicalRegister, result_reg: ClassicalRegister):
        """measure the quantum qubits and store in classic bits"""
        circ.measure(aux_reg, classic_aux_reg)
        circ.measure(b_reg, result_reg)

    def transpile(self, simulator=AerSimulator):
        """function to transpile circuit. required to be executed before run"""
        b_reg, c_reg, aux_reg, classic_aux_reg, result_reg = self._create_register()


        circ = QuantumCircuit(aux_reg, c_reg, b_reg, result_reg, classic_aux_reg)
        self._prepare_state(circ, b_reg)
        self._qpe(circ, b_reg, c_reg)
        circ.barrier()
        self._controlled_rotation(circ, c_reg, aux_reg)
        circ.barrier()
        self._qpe(circ, b_reg, c_reg, inverse=True)
        circ.barrier()
        self._measure(circ, aux_reg,  b_reg, classic_aux_reg, result_reg)

        self.backend = simulator()
        return transpile(circ, backend=self.backend, callback=progress_callback), circ
    
    def run(self, circ, shots=1):
        try:
            return self.backend.run(circ, shots=shots).result()
        except AttributeError:
            raise AttributeError("Circuit is not Transpiled before running")
    
    def transpile_and_run(self, shots=1):
        transpiled_circ = self.transpile()
        return self.run(transpiled_circ, shots=shots).result()
    
    def analyze(self, result: Result) -> tuple[float, np.ndarray]:
        count = result.get_counts()

        if (isinstance(count, dict)):
            # print(count)
            success = {res: cnt for res, cnt in count.items() if res.split()[0] == '1'}
            # failure = {res: cnt for res, cnt in count.items() if res.split()[0] == '0'}
            tot = sum(success.values())
            success_rate = tot / sum(count.values())
            
            x = np.zeros((1 << self.n_b), dtype=float)
            for res, cnt in success.items():
                ind = int(res.split()[1], 2)
                x[ind] = cnt / tot
            return success_rate, x
        else:
            print(count)
            raise TypeError("result.get_counts() is not type dict[str, int]")

In [8]:
# [  1,   0,   0,  0],
#  [ 0,  1,  0,   0],
#  [  0,  0,   1,   0],
#  [ 0,   0,   0,  1]
A = np.array([
[1, 0, 1],
[0, 1, 0],
[1, 0, -1]
], dtype=float)
b = np.array([3, 1, -2], dtype=float)

alg = HHL(A, b)
circ, c = alg.transpile()

A is not power of 2, pad to make A 4 * 4
-----------------------------------
b is not normalized, normalized to b = [ 0.80178373  0.26726124 -0.53452248  0.        ] during init
-----------------------------------


In [9]:
for shots in [10**x for x in range(3, 6)]:
    print(f"shots: {shots}")
    result = alg.run(circ, shots=shots)

    p, x = alg.analyze(result)
    print(p, x, sep='\n')

# classic
x_classic = np.linalg.solve(A, b)
print(x_classic ** 2 / np.linalg.norm(x_classic) ** 2)

shots: 1000
0.019
[0. 1. 0. 0.]
shots: 10000
0.0159
[0. 1. 0. 0.]
shots: 100000
0.01438
[0. 1. 0. 0.]
[0.03333333 0.13333333 0.83333333]
